# Expérience 5.2 (papier) — Comparer les gaps pour un VAE amorti + Extension Contextual Flow (Fashion-MNIST)

Ici, on reproduit l’expérience 5.2 du papier : on entraîne **un VAE complet** (encodeur + décodeur) avec une famille amortie donnée $q_\phi(z|x)$, puis on mesure les trois gaps sur un petit subset fixe.

On compare deux modèles amortis :
- **FFG (Gaussian)** : $q_\phi(z|x)$ est une gaussienne factorisée,
- **Flow** : $q_\phi(z|x)$ est un posterior plus flexible (avec transformations inversibles).
- **Contextual Flow** : même idée, mais les paramètres du flow dépendent explicitement de $x$ (via l’encodeur).  
  On écrit :
  $$
  z = f_{\lambda(x)}(z_0), \qquad z_0 \sim \mathcal N(0,I),
  $$
  avec $\lambda(x)$ produit par le réseau d’inférence.  
  Donc :
  $$
  \log q_\phi(z|x)=\log q(z_0)-\log\left|\det\frac{\partial f_{\lambda(x)}}{\partial z_0}\right|.
  $$
  Donc la forme du posterior peut changer plus finement selon chaque entrée $x$.


La procédure est :
1) On fixe un subset de données pour comparer vite et de façon stable.  
2) Pour chaque modèle amorti (FFG puis Flow), on entraîne le VAE sur tout le train set.  
3) Sur le subset, on estime $\log \hat p(x)$ (IWAE et aussi AIS ; on garde le max, comme dans le papier).  
4) On calcule $\mathcal{L}[q]$ : l’ELBO **amorti** avec l’encodeur appris.  
5) On calcule $\mathcal{L}[q^*]$ en faisant une **optimisation locale** de $q$ pour chaque point (on le fait dans deux familles possibles : Gaussien et Flow).  

Ensuite, on déduit les gaps :
- approximation gap : $\log \hat p(x) - \mathcal{L}[q^*]$  
- amortization gap : $\mathcal{L}[q^*] - \mathcal{L}[q]$  
- inference gap : $\log \hat p(x) - \mathcal{L}[q]$

Ce qu’on veut observer : même si le modèle Flow est plus expressif, est-ce que le gain vient surtout de la **réduction de l’approximation gap**, ou est-ce qu’on réduit aussi l’**amortization gap** (donc l’encodeur généralise mieux) ? 

# Import

In [1]:
import os, sys, numpy as np, csv, time
from pathlib import Path
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from tqdm import tqdm

base_dir = Path.cwd().parent
sys.path.insert(0, str(base_dir / 'models'))
sys.path.insert(0, str(base_dir / 'models' / 'utils'))

from vae_2 import VAE
from inference_net import standard
from distributions import Gaussian, Flow, ContextualFlow
from optimize_local_q import optimize_local_q_dist
from ais3 import test_ais

# Dataset and Device 

In [ ]:
device = torch.device(
    'cuda' if torch.cuda.is_available()
    else 'mps' if torch.backends.mps.is_available()
    else 'cpu'
)
print(f'Device: {device}')


# On charge le dataset Fashion-MNIST depuis un fichier .npz
npz_path = 'fashion_mnist.npz'  
data = np.load(npz_path)

print("Clés:", data.files)

# On récupère les données d'entraînement
train_data = data['X_train']

# On récupère les données de test
test_data  = data['X_test']

print(f"Train: {train_data.shape}, Test: {test_data.shape}")


# Ici, on prend tout le train et tout le test
train_x = train_data           
test_x  = test_data       

# On fixe la dimension de l'entrée (x_size)
# et la dimension latente (z_size)
x_size, z_size = train_x.shape[1], 50

# Définitions des fonctions utiles 
- de train 
- d'évaluations des gaps


In [ ]:
# On désactive le calcul de gradient ici car on est en évaluation (plus rapide, moins de mémoire)
@torch.no_grad()
def estimate_logp_IWAE(model, X, K=10000, batch_size=100):
    # On va stocker la valeur IWAE pour chaque batch puis moyenner
    vals = []
    
    # On crée un DataLoader pour parcourir X par batches
    loader = torch.utils.data.DataLoader(
        torch.from_numpy(X).float(), batch_size=batch_size, shuffle=False
    )
    
    # On boucle sur les batches
    for xb in loader:
        xb = xb.to(device)
        
        # forward2 renvoie l'estimateur IWAE pour k=K samples
        v, _, _ = model.forward2(xb, k=K)
        vals.append(v.item())
        
    # On renvoie la moyenne sur tous les batches
    return float(np.mean(vals))


def estimate_logp_AIS(model, X, K=100, T=500, batch_size=100):
    # Ici on estime log p(x) avec AIS, batch par batch
    vals = []
    
    for i in range(0, len(X), batch_size):
        xb = X[i:i+batch_size]
        try:
            # test_ais renvoie une estimation de log p(x)
            est = test_ais(model, xb, xb.shape[0], 0, K, T)
            vals.append(float(est.item() if torch.is_tensor(est) else est))
        except Exception as e:
            print("AIS batch fail:", e)
    return float(np.mean(vals)) if vals else np.nan


# On désactive le gradient car c'est aussi une évaluation
@torch.no_grad()
def amortized_elbo(model, X, batch_size=100):
    # On calcule la ELBO amortie L[q_phi] sur X
    vals = []
    
    loader = torch.utils.data.DataLoader(
        torch.from_numpy(X).float(), batch_size=batch_size, shuffle=False
    )
    
    for xb in loader:
        xb = xb.to(device)
        
        # forward = ELBO standard avec k=1 sample, warmup=1.0 (KL pleinement actif)
        v, _, _ = model.forward(xb, k=1, warmup=1.0)
        vals.append(v.item())
        
    # On retourne la moyenne sur tous les batches
    return float(np.mean(vals))


def locally_optimized_elbo(model, X, q_star_class, n_points=10):
    # Ici on calcule L[q*] : on optimise localement une q (par point)
    hyper = dict(model.hyper_params)
    
    # On adapte le contexte si on optimise une ContextualFlow
    if q_star_class is ContextualFlow:
        hyper['context_size'] = 128
    else:
        hyper['context_size'] = 0

    # On stocke les ELBO optimisées par point
    vals = []

    # On ne fait pas tous les points (trop long), juste n_points
    for i in tqdm(range(min(n_points, len(X))), desc=f"q* = {q_star_class.__name__}"):
        x = torch.from_numpy(X[i]).float().view(1, -1).to(device)
        
        # On définit log p(x,z) - log q(z|x) via la fonction logposterior du modèle
        logpost = lambda z: model.logposterior_func2(x=x, z=z)

        # On instancie une distribution locale q(z|x) du bon type (Gaussian, Flow, ContextualFlow...)
        q_local = q_star_class(hyper).to(device)

        # On warm-start q_local avec les paramètres de la q amortie du modèle (si possible)
        # strict=False car certaines clés peuvent manquer selon les familles
        try:
            q_local.load_state_dict(model.hyper_params["q"].state_dict(), strict=False)
        except:
            pass

        # On optimise q_local pour ce x (c'est ça qui approxime q*)
        Lqs, _ = optimize_local_q_dist(logpost, hyper, x, q_local)
        vals.append(float(Lqs.item()))

    # On renvoie la moyenne des L[q*] sur les points évalués
    return float(np.mean(vals))


def build_vae(q_class):
    # On prépare la taille du "contexte" uniquement si on est en ContextualFlow
    context_size = 128 if q_class is ContextualFlow else 0
    
    # L'encodeur doit prédire mean/logvar (2*z_size) + éventuellement le contexte
    output_size = 2 * z_size + context_size
    
    # On choisit une architecture différente pour ContextualFlow 
    # car on a déjà plus de flexibilité via le flow conditionné
    if q_class is ContextualFlow:
        enc_arch = [[x_size, 100], [100, output_size]]
    else:
        enc_arch = [[x_size, 200], [200, 200], [200, output_size]]
        
    # Le décodeur reste identique pour comparer proprement
    dec_arch = [[z_size, 200], [200, 200], [200, x_size]]

    # Hyperparamètres du modèle
    hyper = {
        "x_size": x_size, "z_size": z_size,
        "act_func": F.elu,  
        "encoder_arch": enc_arch,
        "decoder_arch": dec_arch,
        "q_dist": standard,
        "cuda": int(device.type == "cuda"),
        "hnf": 0,
        "context_size": context_size
    }

    # On instancie la distribution q 
    q = q_class(hyper)
    hyper["q"] = q
    
    # On construit le VAE complet et on garde hyper_params pour pouvoir refaire q_local plus tard
    m = VAE(hyper).to(device)
    m.hyper_params = hyper
    return m


def train_model(model, X, epochs=700, batch_size=100, lr=1e-3):
    # On prépare le DataLoader pour l'entraînement
    X_t = torch.from_numpy(X).float()
    loader = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(X_t, torch.zeros(len(X))),
        batch_size=batch_size, shuffle=True
    )

    # On entraîne l'encodeur (q_dist) et le décodeur (generator) ensemble
    opt = optim.Adam(
        list(model.q_dist.parameters()) + list(model.generator.parameters()),
        lr=lr
    )

    # Warm-up KL : on augmente progressivement le poids du KL au début
    warm_T = 100.0 
    global_step = 0

    # Boucle d'entraînement classique
    for ep in range(1, epochs+1):
        for xb, _ in loader:
            xb = xb.to(device)
            global_step += 1
            
            # warm augmente de 0 à 1 sur les 100 premières itérations environ
            warm = min(global_step / warm_T, 1.0)

            # forward retourne l'ELBO, on minimise -ELBO
            elbo, _, _ = model.forward(xb, k=1, warmup=warm)
            loss = -elbo

            opt.zero_grad()
            loss.backward()
            opt.step()
        if ep % 50 == 0:
            print(f"[{ep}/{epochs}] ELBO={elbo.item():.3f}")

    # On renvoie le modèle entraîné
    return model

# Définitions de la fonction de run

In [ ]:
def run_exp_5_2(train_x, test_x):

    # On crée un dossier de sortie pour stocker les résultats de l'expérience 5.2
    out_dir = Path("exp52_results_VF_fashion_CFlow")
    out_dir.mkdir(exist_ok=True)
    csv_path = out_dir / "exp52_results_VF_fashion_CFlow.csv"

    # On initialise le CSV une seule fois
    if not csv_path.exists():
        with open(csv_path, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([
                "model_family", "q_star", "logp",
                "Lq", "Lq_star",
                "approx_gap", "amort_gap", "infer_gap"
            ])

    # On définit les trois familles amorties qu'on veut entraîner et comparer
    models = [("FFG", Gaussian), ("Flow", Flow), ("ContextualFlow", ContextualFlow)]

    # On fixe un subset aléatoire mais reproductible (seed=0) pour l'évaluation des gaps
    rng = np.random.default_rng(0)
    subset = train_x[rng.choice(len(train_x), size=15, replace=False)]

    # On boucle sur chaque famille (modèle amorti) : FFG, Flow, ContextualFlow
    for name, q_class in models:

        # On construit le VAE avec la famille q(z|x) choisie, puis on l'entraîne
        model = build_vae(q_class)
        model = train_model(model, train_x, epochs=500)

        # On estime log p(x) avec deux estimateurs : IWAE et AIS
        # Puis on prend le max (comme dans ton setup précédent)
        logp_iwae = estimate_logp_IWAE(model, subset, K=10000)
        logp_ais  = estimate_logp_AIS(model, subset, K=100, T=500)

        logp = max(logp_iwae, logp_ais if not np.isnan(logp_ais) else -1e9)

        # On calcule la borne amortie L[q] (avec l'encodeur appris)
        Lq = amortized_elbo(model, subset)

        # On calcule ensuite les bornes localement optimisées L[q*] pour trois familles possibles
        # (même si le modèle amorti n'est pas de cette famille)
        Lqs_FFG = locally_optimized_elbo(model, subset, Gaussian)
        Lqs_Flow = locally_optimized_elbo(model, subset, Flow)
        Lqs_CFlow = locally_optimized_elbo(model, subset, ContextualFlow)

        # On calcule les trois gaps à partir de (logp, Lq, Lq*)
        # approx_gap = logp - Lq*
        # amort_gap  = Lq* - Lq
        # infer_gap  = logp - Lq
        def gaps(logp, Lq, Lqs):
            return round(logp - Lqs,2), round(Lqs - Lq,2), round(logp - Lq,2)

        g_FFG  = gaps(logp, Lq, Lqs_FFG)
        g_Flow = gaps(logp, Lq, Lqs_Flow)
        g_CFlow = gaps(logp, Lq, Lqs_CFlow)

        # On affiche un résumé pour vérifier que tout est cohérent
        print("\n=== Résultats ===")
        print(f"log p̂(x)     = {logp:.3f}")
        print(f"L[q]         = {Lq:.3f}")
        print(f"L[q*_FFG]    = {Lqs_FFG:.3f}")
        print(f"L[q*_Flow]   = {Lqs_Flow:.3f}")
        print(f"L[q*_CFlow]  = {Lqs_CFlow:.3f}")
        print(f"Gaps FFG     = {g_FFG}")
        print(f"Gaps Flow    = {g_Flow}")
        print(f"Gaps CFlow   = {g_CFlow}")

        # On sauvegarde les résultats dans le CSV :
        # 3 lignes par modèle (q* = FFG, Flow, ContextualFlow)
        with open(csv_path, "a", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([name, "FFG",  round(logp,2), round(Lq,2), round(Lqs_FFG,2),  *g_FFG])
            writer.writerow([name, "Flow", round(logp,2), round(Lq,2), round(Lqs_Flow,2), *g_Flow])
            writer.writerow([name, "ContextualFlow", round(logp,2), round(Lq,2), round(Lqs_CFlow,2), *g_CFlow])

run_exp_5_2(train_x, test_x)
print(" Résultats sauvés dans exp52_results_VF_fashion_CFlow/exp52_results_VF_fashion_CFlow.csv")